# Desafío Complementario: CrossValidation y mejora de modelos de ML

In [22]:
# Importacion de librerias para manipulacion de datos
import pandas as pd
import numpy as np

In [23]:
# Importacion de Plotly como libreria de DataViz
import plotly.graph_objects as go
import plotly.figure_factory as ff

In [24]:
# Importacion de Sklearn para la costruccion de modelo
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_curve, auc
from sklearn.preprocessing import MinMaxScaler

#### Configurando path

In [25]:
import os

# Obtiene la ubicación completa del archivo del notebook
notebook_path = os.path.realpath('__file__')

# Obtiene la ruta absoluta de la carpeta "Coderhouser-DS-TP0002" sin importar donde te encuentres
path_base = os.path.abspath(os.path.join(notebook_path, "..", "..", "..", "Coderhouser-DS-TP0002"))

print("Path base':", path_base)

Path base': C:\develoment\Coderhouser-DS-TP0002


#### Leyendo datos:

In [26]:
# Generando el Dataframe apartir de los dataset RedTeam y BlueTeam
blueTeam = pd.read_csv( path_base + r"\\data\procesed\blueTeam.csv")
redTeam = pd.read_csv( path_base + r"\\data\procesed\redTeam.csv")
data = pd.concat([blueTeam,redTeam])

In [27]:
# Crear un mapeo de colores
color_map = {'Win': 'yellow', 'Loss': 'gray'}
data.drop(columns=["Unnamed: 0"], inplace=True)

### Preparar datos:


In [28]:

X = data.drop('Wins', axis=1)  
y = data['Wins']


In [29]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


### Crear modelo de Random Forest:

In [30]:
# Entrenar un modelo de Random Forest en los datos transformados por PCA
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [31]:
# Realizar predicciones en el conjunto de prueba
y_pred = rf_model.predict(X_test)

In [32]:
# Imprimir matriz de confusión
cm = confusion_matrix(y_test, y_pred)

In [33]:

# Crear la matriz de confusión interactiva con Plotly
fig = ff.create_annotated_heatmap(z=cm, x=['Predicción Negativa', 'Predicción Positiva'], y=['Real Negativa', 'Real Positiva'], colorscale='Viridis')

# Diseño del gráfico
fig.update_layout(
    title='Matriz de Confusión',
    xaxis=dict(title='Predicción'),
    yaxis=dict(title='Realidad'),
)

# Mostrar la figura interactiva
fig.show()

### Implementar feature_importances

In [45]:
# Obtener las importancias de características
feature_importances = rf_model.feature_importances_

# Sumar las importancias de características
total_importance = sum(feature_importances)

# Imprimir las importancias individuales y la suma total
print('Importancias de Características:')
for feature, importance in zip(X.columns, feature_importances):
    print(f'{feature}: {importance}')

print('\nSuma Total de Importancias:', total_importance)

Importancias de Características:
WardsPlaced: 0.04958898176554363
WardsDestroyed: 0.034031376894272616
FirstBlood: 0.010648772628955638
Kills: 0.035516436692723446
Deaths: 0.0505419083825405
Assists: 0.04220518533141124
EliteMonsters: 0.013474100703412572
Dragons: 0.011620343415851863
Heralds: 0.0063083786621370656
TowersDestroyed: 0.002217302914437313
TotalGold: 0.0813581321261176
AvgLevel: 0.030060723618775285
TotalExperience: 0.08416999722577966
TotalMinionsKilled: 0.05238085284346628
TotalJungleMinionsKilled: 0.0525142715704186
GoldDiff: 0.1603230763394784
ExperienceDiff: 0.13981189273231204
CSPerMin: 0.05198505098098863
GoldPerMin: 0.09124321517137762

Suma Total de Importancias: 1.0


### Aplicar validación cruzada:

In [34]:
# Utiliza el número de divisiones y la métrica deseada
cv_scores = cross_val_score(rf_model, X, y, cv=5, scoring='accuracy')  

In [40]:
print(f'Puntuaciones de Validación Cruzada: {cv_scores}')
print(f'Precisión Promedio: {cv_scores.mean()}')

Puntuaciones de Validación Cruzada: [0.72039474 0.72165992 0.71761134 0.71829916 0.72285497]
Precisión Promedio: 0.7201640260191393


In [41]:
# Visualizar resultados con Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(1, 6)), y=cv_scores, mode='lines+markers', name='Puntuaciones de Validación Cruzada'))

fig.update_layout(
    title='Puntuaciones de Validación Cruzada para Random Forest',
    xaxis_title='Iteración de Validación Cruzada',
    yaxis_title='Precisión',
    showlegend=True,
)

fig.show()

### Implementar RandomizedSearchCV

In [36]:
# Definir la cuadrícula de hiperparámetros para la búsqueda aleatoria
param_dist = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Crear un objeto RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf_model, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', random_state=42)

# Ajustar el modelo a los datos
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
                   param_distributions={'max_depth': [None, 10, 20, 30],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 100, 200, 300]},
                   random_state=42, scoring='accuracy')

In [37]:
# Obtener el mejor modelo y los mejores parámetros
best_model = random_search.best_estimator_
best_params = random_search.best_params_

In [38]:
# Realizar predicciones en el conjunto de prueba con el mejor modelo
y_pred = best_model.predict(X_test)

In [39]:
# Imprimir la precisión del modelo con los mejores parámetros
accuracy = accuracy_score(y_test, y_pred)
print('Precisión del modelo con los mejores parámetros:', accuracy)

# Imprimir los mejores parámetros encontrados por RandomizedSearchCV
print('Mejores Parámetros:', best_params)

Precisión del modelo con los mejores parámetros: 0.7280701754385965
Mejores Parámetros: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_depth': 10}


### Aplicando Curva de ROC

In [42]:
# Obtener probabilidades de predicción para la clase positiva
y_probs = rf_model.predict_proba(X_test)[:, 1]

In [43]:
# Calcular la curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_probs)
roc_auc = auc(fpr, tpr)

In [44]:
# Crear la figura interactiva de Plotly
fig = go.Figure()

# Agregar la curva ROC
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f'Curva ROC (AUC = {roc_auc:.2f})'))

# Línea de referencia
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Línea de Referencia', line=dict(dash='dash')))

# Diseño del gráfico
fig.update_layout(
    title='Curva ROC con Random Forest',
    xaxis_title='Tasa de Falsos Positivos (FPR)',
    yaxis_title='Tasa de Verdaderos Positivos (TPR)',
    legend=dict(x=0.02, y=0.95),
    margin=dict(l=0, r=0, t=30, b=0)
)

# Mostrar la figura interactiva
fig.show()